In [1]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import pandas as pd
from cloudy_fit_lib import *
from scipy.interpolate import RegularGridInterpolator
import pickle

In [2]:
plt.style.use('/Users/thepoetoftwilight/Documents/CUBS/Code/science.mplstyle')

In [12]:
rootdir = '/Users/thepoetoftwilight/Documents/CUBS/Data/PG1522+101/cloudy_grids/ext_grid/'

Obtain the UVB grid from FG20

In [13]:
fg20_z_grid, fg20_wav_grid, fg20_J_nu_grid = read_uvb('', 'fg20_galaxy.ascii')

Fetch the SED at $z=1$

In [14]:
z_test = 1

In [15]:
fg20_J_nu_test = fetch_sed(z_test, fg20_z_grid, fg20_J_nu_grid)

Specify grid points

In [16]:
# Discrete grid points
grid_points = np.array([[16.  , -4.25,  1.  ],
       [16.25, -5.  ,  1.  ],
       [16.25, -4.75,  1.  ],
       [16.25, -4.5 ,  1.  ],
       [16.25, -4.25,  1.  ],
       [16.5 , -5.  ,  0.75],
       [16.5 , -5.  ,  1.  ],
       [16.5 , -4.75,  0.75],
       [16.5 , -4.75,  1.  ],
       [16.5 , -4.5 ,  1.  ],
       [16.5 , -4.25,  1.  ],
       [16.75, -5.  ,  0.75],
       [16.75, -5.  ,  1.  ],
       [16.75, -4.75,  0.75],
       [16.75, -4.75,  1.  ],
       [16.75, -4.5 ,  0.75],
       [16.75, -4.5 ,  1.  ],
       [16.75, -4.25,  1.  ],
       [17.  , -5.  ,  0.75],
       [17.  , -5.  ,  1.  ],
       [17.  , -4.75,  0.75],
       [17.  , -4.75,  1.  ],
       [17.  , -4.5 ,  0.75],
       [17.  , -4.5 ,  1.  ],
       [17.  , -4.25,  1.  ]])

Process the grid to identify "failures"

In [17]:
log_T_pie_thresh = 4

In [18]:
# Bifurcate failures based on stopping temperature
low_temp_failures = []
high_temp_failures = []

In [23]:
for i in range(len(grid_points)):
    
    logN_HI, log_hdens, log_metals = grid_points[i]
    
    # The filename corresponding to this grid point
    
    logN_HI_str = '{:.2f}'.format(logN_HI).replace('.','')
    log_hdens_str = '{:.2f}'.format(log_hdens).replace('.','')
    log_metals_str = '{:.2f}'.format(log_metals).replace('.','')
    
    filename = 'igm_lalpha_hm12_grid_{}_{}_{}'.format(logN_HI_str, log_hdens_str, log_metals_str)
    
    # Get average (log) HI temperatures for all grid points
    log_temp = read_avr_file(rootdir, filename)[0]
    
    # Get column densities for all species
    species_names, log_col_dens_grid = read_col_file(rootdir, filename)
    log_col_dens = log_col_dens_grid[0]

    # Check if this grid point is a failure
    if np.round(log_col_dens[0],2) != logN_HI:
        # Categorize based on temperature
        if log_temp<log_T_pie_thresh:
            low_temp_failures.append([logN_HI, log_hdens, log_metals, log_temp, log_col_dens[0]])
        else:
            high_temp_failures.append([logN_HI, log_hdens, log_metals, log_temp, log_col_dens[0]])

In [24]:
len(low_temp_failures)

0

In [25]:
len(high_temp_failures)

0